In [2]:
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Initialize WordNet Lemmatizer for word normalization
lemmatizer = WordNetLemmatizer()
# Load intents from JSON file
intents = json.loads(open('intents.json').read())

In [4]:
# Extract unique tags from intents
tags = []
for entry in intents['intents']:
    tags.append(entry['tag'])
        

In [5]:
#show tags
tags

['greeting',
 'goodbye',
 'thanks',
 'stocks',
 'weather',
 'help',
 'fallback',
 'small_talk',
 'hobbies',
 'compliments',
 'day',
 'movies',
 'technology',
 'personal',
 'books',
 'food',
 'travel',
 'pets',
 'learning',
 'health',
 'nature',
 'music',
 'books',
 'art',
 'career',
 'languages',
 'movies',
 'creativity',
 'mindfulness',
 'fashion',
 'positivity',
 'cooking',
 'relationships',
 'environment',
 'selfcare',
 'productivity',
 'mindfulness',
 'sports',
 'education',
 'finance',
 'leadership',
 'technology_trends',
 'mindfulness_benefits',
 'space_exploration',
 'nutrition',
 'career_growth',
 'parenting_challenges',
 'education_technology',
 'mental_health',
 'home_decor',
 'personal_growth',
 'entrepreneurship',
 'philosophy',
 'fitness',
 'social_media',
 'cultural_diversity',
 'personal_finances',
 'gardening',
 'time_management',
 'aging_gracefully',
 'volunteering',
 'parenting_styles',
 'gossip_family',
 'gossip_friends',
 'daily_life_chatter',
 'celebrity_gossip',
 

In [6]:
# Initialize lists to hold words, classes, and training data
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

# Tokenize patterns from intents and preprocess data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)  # Use extend to add words into word_list
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [7]:
# Normalize words and remove ignored characters
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [8]:
classes = sorted(set(classes))

In [9]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [10]:
# Prepare training data
training = []
output_empty = [0] * len(classes)

In [11]:
# Iterate through each document in the list of documents
for document in documents:
    bag = []    # Initialize an empty bag to hold word occurrence information
    word_patterns = document[0]     # Extract the word patterns from the document
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]      # Normalize and lemmatize words
    
    # Iterate through each word in the vocabulary (all unique words in the dataset)
    for word in words:
        # Check if the current word exists in the word patterns
        # If it does, add 1 to the bag to represent word occurrence, otherwise add 0
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)# Initialize an empty list for the output row
    # Set the corresponding index in the output_row to 1 to represent the intent class
    output_row[classes.index(document[1])] = 1
    # Append the bag and output_row as a training instance to the training list
    training.append([bag, output_row])

In [12]:
# Shuffle and convert training data to numpy array
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Justi\AppData\Local\Temp\ipykernel_7980\2858459658.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [13]:
# Build the neural network model
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [15]:
# Configure Adam optimizer
adam_optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
# Add learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-6)
callbacks = [lr_scheduler]

In [16]:
# Train the model
model.fit(np.array(train_x),np.array(train_y), epochs = 600, batch_size = 2, verbose=1)

Epoch 1/600
316/316 [==============================] - 1s 1ms/step - loss: 4.9779 - accuracy: 0.0095
Epoch 2/600
316/316 [==============================] - 0s 990us/step - loss: 4.8140 - accuracy: 0.0396
Epoch 3/600
316/316 [==============================] - 0s 987us/step - loss: 4.4832 - accuracy: 0.0666
Epoch 4/600
316/316 [==============================] - 0s 987us/step - loss: 3.9354 - accuracy: 0.1506
Epoch 5/600
316/316 [==============================] - 0s 997us/step - loss: 3.4754 - accuracy: 0.2155
Epoch 6/600
316/316 [==============================] - 0s 1ms/step - loss: 2.9655 - accuracy: 0.3344
Epoch 7/600
316/316 [==============================] - 0s 994us/step - loss: 2.5372 - accuracy: 0.3930
Epoch 8/600
316/316 [==============================] - 0s 1ms/step - loss: 2.2726 - accuracy: 0.4358
Epoch 9/600
316/316 [==============================] - 0s 994us/step - loss: 1.8585 - accuracy: 0.5357
Epoch 10/600
316/316 [==============================] - 0s 1ms/step - loss: 1.6

In [17]:
# Save the trained model
model.save('chatbot_model.model')

INFO:tensorflow:Assets written to: chatbot_model.model\assets


INFO:tensorflow:Assets written to: chatbot_model.model\assets


In [18]:
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [19]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [20]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.model')

In [1]:
import sys
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextBrowser, QLineEdit, QPushButton, QVBoxLayout, QWidget, QHBoxLayout

filename = 'intents.json'

class ChatBotGUI(QMainWindow):
    def __init__(self):
        super().__init__()

        self.initUI()

        self.lemmatizer = WordNetLemmatizer()
        self.intents = json.loads(open(filename).read())

        self.words = pickle.load(open('words.pkl', 'rb'))
        self.classes = pickle.load(open('classes.pkl', 'rb'))
        self.model = load_model('chatbot_model.model')

    def initUI(self):
        self.setGeometry(100, 100, 400, 600)
        self.setWindowTitle('ChatBot GUI')

        self.text_browser = QTextBrowser(self)
        self.user_input = QLineEdit(self)
        self.send_button = QPushButton('Send', self)

        layout = QVBoxLayout()
        layout.addWidget(self.text_browser)

        input_layout = QHBoxLayout()  
        input_layout.addWidget(self.user_input)
        input_layout.addWidget(self.send_button)
        layout.addLayout(input_layout) 

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        self.send_button.clicked.connect(self.on_send_button_click)
        self.user_input.returnPressed.connect(self.send_button.click)

        # Apply styles using QSS
        with open('style.qss', 'r') as style_file:
            self.setStyleSheet(style_file.read())

    def clean_up_sentence(self, sentence):
        sentence_words = nltk.wordpunct_tokenize(sentence)
        sentence_words = [self.lemmatizer.lemmatize(word) for word in sentence_words]
        return sentence_words

    def bag_of_words(self, sentence):
        sentence_words = self.clean_up_sentence(sentence)
        bag = [0] * len(self.words)
        for w in sentence_words:
            for i, word in enumerate(self.words):
                if word == w:
                    bag[i] = 1
        return np.array(bag)

    def predict_class(self, sentence):
        bow = self.bag_of_words(sentence)
        res = self.model.predict(np.array([bow]))[0]
        ERROR_THRESHOLD = 0.25
        result = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

        result.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in result:
            return_list.append({'intent': self.classes[r[0]], 'probability': str(r[1])})
        return return_list

    def get_response(self, intents_list, intents_json):
        tag = intents_list[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result

    def on_send_button_click(self):
        user_message = self.user_input.text()
        self.user_input.clear()

        ints = self.predict_class(user_message)
        response = self.get_response(ints, self.intents)

        self.text_browser.append("<span style='color: blue;'>You: </span>" + user_message)
        self.text_browser.append("<span style='color: green;'>Bot: </span>" + response)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    gui = ChatBotGUI()
    gui.show()
    sys.exit(app.exec_())


1/1 [==============================] - 0s 11ms/step
